下面两个文件。prepare_data.py是用来提供训练数据的，其中把有标记的train分为三份，一份全是正标签，一份全是负标签，一份是所有的。这三份都经过了简单的预处理，比如用BeautifulSoup去html tag，以及去`”`号，去`\\`号。

而generate_result.py用于生成用于提交的结果。结果真正用于训练word2vec和预测的部分，全都使用C语言写的……

我打算自己重写这一部分，这里列一些比较好的文章，[A gentle introduction to Doc2Vec](https://medium.com/scaleabout/a-gentle-introduction-to-doc2vec-db3e8c0cce5e)。

# prepare_data.py

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
train_data = pd.read_csv("../Sentiment/data/labeledTrainData.tsv", header=0, 
                         delimiter='\t', quoting=3, error_bad_lines=False)

train_data.shape

(25000, 3)

In [4]:
train_data.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [7]:
one_row = train_data['review'][0]
one_row

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [8]:
one_row = BeautifulSoup(one_row, 'lxml').get_text()
one_row

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 2

In [9]:
# 去除“符号
one_row = one_row.strip('"')
one_row

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

In [10]:
one_row = one_row.replace('\\', '')
one_row

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

In [11]:
bw_pos = open('full-train-pos.txt', 'w')
bw_neg = open('full-train-neg.txt ', 'w')

bw_train = open('full-train.txt', 'w')

num_reviews = train_data["review"].size
print("Cleaning and dumping the training set movie reviews...")
for i in range(0, num_reviews):
    text = BeautifulSoup(train_data["review"][i], 'lxml').get_text()

    text = text.strip('"')
    text = text.replace('\\', '')
    
    label = train_data["sentiment"][i]
    if label == 1:
        bw_pos.write(text + '\n')
    else:
        bw_neg.write(text + '\n')
    
    bw_train.write(text + '\n')

bw_pos.close()
bw_neg.close()
bw_train.close()

Cleaning and dumping the training set movie reviews...


In [13]:
test_data = pd.read_csv("../Sentiment/data/testData.tsv", header = 0, delimiter = "\t", quoting = 3)

num_reviews = len(test_data["review"])
bw_test = open('full-test.txt', 'w')

print("Cleaning and dumping the testing set movie reviews...")
for i in range(0, num_reviews):
    text = BeautifulSoup(test_data["review"][i], 'lxml').get_text()

    text = text.strip('"')
    text = text.replace('\\', '')
    
    bw_test.write(text + '\n')
bw_test.close()

Cleaning and dumping the testing set movie reviews...


In [14]:
unlabel_data = pd.read_csv("../Sentiment/data/unlabeledTrainData.tsv", header = 0, delimiter = "\t", quoting = 3)
num_reviews = len(test_data["review"])
bw_unlabel = open('full-unlabeled.txt', 'w')

print("Cleaning and dumping the unlabeled set movie reviews...")
for i in range(0, num_reviews):
    text = BeautifulSoup(unlabel_data["review"][i], 'lxml').get_text()
    text = text.strip('"')
    text = text.replace('\\', '')
    
    bw_unlabel.write(text + '\n')
bw_unlabel.close()

Cleaning and dumping the unlabeled set movie reviews...


# generate_result.py

In [ ]:
import pandas as pd

test_data = pd.read_csv("../Sentiment/data/testData.tsv", header = 0, delimiter = "\t", quoting = 3)
num_reviews = len(test_data["id"])

br_result = open('PARAGRAPH-TEST', 'r')
result_list = []

for line in br_result:
    line = line.strip()
    strs = line.split()
    result_list.append(float(strs[1]))
br_result.close()

out = open("result\\doc2vec.csv", 'w')
out.write("\"id\"" + "," + "\"sentiment\"")
out.write("\n")
for i in xrange(num_reviews):
    out.write(test_data["id"][i] + "," + str(result_list[i]) + "\n")
out.close()